# ARX


In [2]:
import numpy as np

from lib.read_data import cycles, u_desvio, y, y_desvio, y_ref

model_name = "ARX"


In [3]:
import hickle as hkl


def gerar_ARX(y, u):
    # Construindo a matriz de regressão
    X = np.hstack([y[:-1, :], u[1:, :]])

    # Saídas futuras
    Y = y[1:, :]

    # Estimando parâmetros via mínimos quadrados
    theta = np.linalg.lstsq(X, Y)[0]
    return theta


def inferir_ARX(theta, y, u):
    N = len(y)
    y_arx = np.zeros_like(y)
    y_arx[0] = y[0]  # inicializa com o primeiro valor real

    for k in range(1, N):
        phi = np.concatenate([y_arx[k - 1], u[k]])
        y_arx[k] = phi @ theta
    return y_arx


theta = gerar_ARX(y_desvio, u_desvio)
hkl.dump(theta, f"../export/{model_name}.hkl")

# print("Parâmetros estimados:\n", theta)
# from lib.read_data import u_ref
# u_sp = np.array([610, 220, 135, 105]) - u_ref
# u_desvio = np.full((200, 4), u_sp)

y_arx = inferir_ARX(theta, y_desvio, u_desvio)

# Convertendo o valor desvio em variavel de engenharia
y_arx = y_arx + np.tile(y_ref, (y_arx.shape[0], 1))


In [4]:
# Plotando resultados
from lib.plot import plot_comparison

plot_comparison(cycles, y, y_arx, model_name)
